In [1]:

import os
import numpy as np
import pandas as pd
import polars as pl
import json
import math
import random
import joblib
from copy import deepcopy
from pathlib import Path
from typing import Tuple, List, Optional
import warnings
warnings.filterwarnings("ignore")
# ML utilities
from sklearn.model_selection import StratifiedGroupKFold, PredefinedSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from lightgbm import LGBMClassifier, log_evaluation, early_stopping
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import clone
import xgboost as xgb
xgb.set_config(verbosity=0)

# World coordinate transformation
from scipy.spatial.transform import Rotation as R

# Competition specific
import kaggle_evaluation.cmi_inference_server
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

print("✓ All imports loaded successfully")
class Config: 
    """Central configuration class for training and data parameters"""
    
    # Paths for Kaggle environment
    EXPORT_DIR = "/kaggle/input/lstm-triet"
    TRAIN_PATH = "/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv"
    TRAIN_DEMOGRAPHICS_PATH = "/kaggle/input/cmi-detect-behavior-with-sensor-data/train_demographics.csv"
    TEST_PATH = "/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv"
    TEST_DEMOGRAPHICS_PATH = "/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv"
    
    # Training parameters
    SEED = 42
    N_FOLDS = 5
    
    # Feature columns
    ACC_COLS = ['acc_x', 'acc_y', 'acc_z']
    ROT_COLS = ['rot_w', 'rot_x', 'rot_y', 'rot_z']
    THM_COLS = ['thm_1', 'thm_2', 'thm_3', 'thm_4', "thm_5"]
    
   

# Set reproducibility
np.random.seed(Config.SEED)

✓ All imports loaded successfully


In [2]:
GESTURE_MAPPER = {
    "Above ear - pull hair": 0,
    "Cheek - pinch skin": 1,
    "Eyebrow - pull hair": 2,
    "Eyelash - pull hair": 3, 
    "Forehead - pull hairline": 4,
    "Forehead - scratch": 5,
    "Neck - pinch skin": 6, 
    "Neck - scratch": 7,
    
    "Drink from bottle/cup": 8,
    "Feel around in tray and pull out an object": 9,
    "Glasses on/off": 10,
    "Pinch knee/leg skin": 11, 
    "Pull air toward your face": 12,
    "Scratch knee/leg skin": 13,
    "Text on phone": 14,
    "Wave hello": 15,
    "Write name in air": 16,
    "Write name on leg": 17,
}

REVERSE_GESTURE_MAPPER = {v: k for k, v in GESTURE_MAPPER.items()}

In [3]:
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=8):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction), nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel), nn.Sigmoid())
    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

class ResidualSEBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, drop=0.3):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding="same", bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding="same", bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.se = SEBlock(out_channels)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drop)
        self.pool = nn.MaxPool1d(2)
        self.shortcut = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm1d(out_channels)
        ) if in_channels != out_channels else nn.Identity()
    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.se(self.bn2(self.conv2(out)))
        out = self.dropout(self.pool(self.relu(out + identity)))
        return out

class AttentionLayer(nn.Module):
    def __init__(self, hidden_dim): super().__init__(); self.attn = nn.Sequential(nn.Linear(hidden_dim, 1), nn.Tanh())
    def forward(self, x):
        weights = F.softmax(self.attn(x).squeeze(-1), dim=1).unsqueeze(-1)
        return torch.sum(x * weights, dim=1)

class CrossAttention(nn.Module):
    def __init__(self, feature_dim, num_heads=8, dropout=0.1):
        super().__init__()
        self.feature_dim = feature_dim
        self.num_heads = num_heads
        self.head_dim = feature_dim // num_heads
        assert feature_dim % num_heads == 0, "feature_dim must be divisible by num_heads"
        self.q_linear = nn.Linear(feature_dim, feature_dim, bias=False)
        self.k_linear = nn.Linear(feature_dim, feature_dim, bias=False)
        self.v_linear = nn.Linear(feature_dim, feature_dim, bias=False)
        self.out_linear = nn.Linear(feature_dim, feature_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(feature_dim)
    def forward(self, query_branch, key_value_branches):
        B, T, C = query_branch.shape
        Q = self.q_linear(query_branch)
        all_kv = torch.cat(key_value_branches, dim=1)
        K = self.k_linear(all_kv)
        V = self.v_linear(all_kv)
        Q = Q.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        K = K.view(B, -1, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(B, -1, self.num_heads, self.head_dim).transpose(1, 2)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        attn_output = torch.matmul(attn_weights, V)
        attn_output = attn_output.transpose(1, 2).contiguous().view(B, T, C)
        output = self.out_linear(attn_output)
        output = self.layer_norm(query_branch + output)
        return output

class IMUCrossAttentionFusion(nn.Module):
    def __init__(self, feature_dim=256, num_heads=8, dropout=0.1):
        super().__init__()
        self.cross_attn1 = CrossAttention(feature_dim, num_heads, dropout)
        self.cross_attn2 = CrossAttention(feature_dim, num_heads, dropout)
        self.cross_attn3 = CrossAttention(feature_dim, num_heads, dropout)
    def forward(self, imu1, imu2, imu3):
        e1 = self.cross_attn1(imu1, [imu2, imu3])
        e2 = self.cross_attn2(imu2, [imu1, imu3])
        e3 = self.cross_attn3(imu3, [imu1, imu2])
        return e1, e2, e3

class IMUCrossAttentionModel(nn.Module):
    def __init__(self, imu_dim, n_classes):
        super().__init__()
        self.imu_dim = imu_dim
        # 3 nhánh CNN cho IMU
        self.imu_branch1 = nn.Sequential(ResidualSEBlock(12, 128, 3), ResidualSEBlock(128, 256, 5))
        self.imu_branch2 = nn.Sequential(ResidualSEBlock(11, 128, 3), ResidualSEBlock(128, 256, 5))
        self.imu_branch3 = nn.Sequential(ResidualSEBlock(12, 128, 3), ResidualSEBlock(128, 256, 5))
        self.cross_attention_fusion = IMUCrossAttentionFusion(feature_dim=256)
        self.bilstm = nn.LSTM(256*3, 512, num_layers=1, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.4)
        self.attention = AttentionLayer(1024)
        self.fc_layers = nn.Sequential(
            nn.Linear(1024, 512, bias=False), nn.BatchNorm1d(512), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(512, 256, bias=False), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, n_classes)
        )
    def forward(self, x):
        features = self.extract_features(x)  # call shared feature extractor
        out = self.fc_layers(features)
        return out
        
    def extract_features(self, x):
        imu = x[:, :, :self.imu_dim]
        imu1 = self.imu_branch1(imu[:, :, :12].transpose(1, 2)).transpose(1, 2)
        imu2 = self.imu_branch2(imu[:, :, 12:23].transpose(1, 2)).transpose(1, 2)
        imu3 = self.imu_branch3(imu[:, :, 23:].transpose(1, 2)).transpose(1, 2)
        imu1, imu2, imu3 = self.cross_attention_fusion(imu1, imu2, imu3)
        merged = torch.cat((imu1, imu2, imu3), dim=2)
        lstm_out, _ = self.bilstm(merged)
        lstm_out = self.dropout(lstm_out)
        attended = self.attention(lstm_out)
        return attended  # shape: (batch_size, 1024)

class ModelEMA(nn.Module):
    def __init__(self, model, decay=0.99, device=None):
        super().__init__()
        self.module = deepcopy(model)
        self.module.eval()
        self.decay = decay
        self.device = device
        if self.device:
            self.module.to(device=device)
    def _update(self, model, update_fn):
        with torch.no_grad():
            for ema_v, model_v in zip(self.module.state_dict().values(), model.state_dict().values()):
                if self.device:
                    model_v = model_v.to(device=self.device)
                ema_v.copy_(update_fn(ema_v, model_v))
    def update(self, model): self._update(model, update_fn=lambda e, m: self.decay * e + (1. - self.decay) * m)
    def set(self, model): self._update(model, update_fn=lambda e, m: m)

In [4]:
def compute_world_acceleration(acc: np.ndarray, rot: np.ndarray) -> np.ndarray:
    """
    Convert acceleration from device coordinates to world coordinates
    
    This is the key innovation: normalizing for device orientation
    
    Args:
        acc: acceleration in device coordinates, shape (time_steps, 3) [x, y, z]
        rot: rotation quaternion, shape (time_steps, 4) [w, x, y, z] (normalized)
    
    Returns:
        acc_world: acceleration in world coordinates, shape (time_steps, 3)
        
    Why this matters:
    - Device acceleration depends on how the watch is oriented on the wri/st
    - World acceleration is independent of device orientation
    - This helps the model focus on actual hand motion rather than wrist rotation
    """
    try:
        # Convert quaternion format from [w, x, y, z] to [x, y, z, w] for scipy
        rot_scipy = rot[:, [1, 2, 3, 0]]
        
        # Verify quaternions are valid (non-zero norm)
        norms = np.linalg.norm(rot_scipy, axis=1)
        if np.any(norms < 1e-8):
            # Replace problematic quaternions with identity
            mask = norms < 1e-8
            rot_scipy[mask] = [0.0, 0.0, 0.0, 1.0]  # Identity quaternion in scipy format
        
        # Create rotation object and apply transformation
        r = R.from_quat(rot_scipy)
        acc_world = r.apply(acc)
        
    except Exception:
        # Fallback to original acceleration if transformation fails
        print("Warning: World coordinate transformation failed, using device coordinates")
        acc_world = acc.copy()
    
    return acc_world

In [5]:
#bỏ gia tốc trọng trường
def remove_gravity_from_acc(acc_data: np.ndarray, rot_data: np.ndarray) -> np.ndarray:

    if isinstance(acc_data, pd.DataFrame):
        acc_values = acc_data[['acc_x', 'acc_y', 'acc_z']].values
    else:
        acc_values = acc_data

    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = acc_values.shape[0]
    linear_accel = np.zeros_like(acc_values)
    
    gravity_world = np.array([0, 0, 9.81])

    for i in range(num_samples):
        if np.all(np.isnan(quat_values[i])) or np.all(np.isclose(quat_values[i], 0)):
            linear_accel[i, :] = acc_values[i, :] 
            continue

        try:
            rotation = R.from_quat(quat_values[i])
            gravity_sensor_frame = rotation.apply(gravity_world, inverse=True)
            linear_accel[i, :] = acc_values[i, :] - gravity_sensor_frame
        except ValueError:
             linear_accel[i, :] = acc_values[i, :]
             
    return linear_accel

In [6]:
#tính vận tốc quay
def calculate_angular_velocity_from_quat(rot_data, time_delta=1/200): # Assuming 200Hz sampling rate
    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_vel = np.zeros((num_samples, 3))

    for i in range(num_samples - 1):
        q_t = quat_values[i]
        q_t_plus_dt = quat_values[i+1]

        if np.all(np.isnan(q_t)) or np.all(np.isclose(q_t, 0)) or \
           np.all(np.isnan(q_t_plus_dt)) or np.all(np.isclose(q_t_plus_dt, 0)):
            continue

        try:
            rot_t = R.from_quat(q_t)
            rot_t_plus_dt = R.from_quat(q_t_plus_dt)

            # Calculate the relative rotation
            delta_rot = rot_t.inv() * rot_t_plus_dt
            
            # Convert delta rotation to angular velocity vector
            # The rotation vector (Euler axis * angle) scaled by 1/dt
            # is a good approximation for small delta_rot
            angular_vel[i, :] = delta_rot.as_rotvec() / time_delta
        except ValueError:
            # If quaternion is invalid, angular velocity remains zero
            pass
            
    return angular_vel

In [7]:
def calculate_angular_distance(rot_data):
    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_dist = np.zeros(num_samples)

    for i in range(num_samples - 1):
        q1 = quat_values[i]
        q2 = quat_values[i+1]

        if np.all(np.isnan(q1)) or np.all(np.isclose(q1, 0)) or \
           np.all(np.isnan(q2)) or np.all(np.isclose(q2, 0)):
            angular_dist[i] = 0 # Или np.nan, в зависимости от желаемого поведения
            continue
        try:
            # Преобразование кватернионов в объекты Rotation
            r1 = R.from_quat(q1)
            r2 = R.from_quat(q2)

            # Вычисление углового расстояния: 2 * arccos(|real(p * q*)|)
            # где p* - сопряженный кватернион q
            # В scipy.spatial.transform.Rotation, r1.inv() * r2 дает относительное вращение.
            # Угол этого относительного вращения - это и есть угловое расстояние.
            relative_rotation = r1.inv() * r2
            
            # Угол rotation vector соответствует угловому расстоянию
            # Норма rotation vector - это угол в радианах
            angle = np.linalg.norm(relative_rotation.as_rotvec())
            angular_dist[i] = angle
        except ValueError:
            angular_dist[i] = 0 # В случае недействительных кватернионов
            pass
            
    return angular_dist

In [8]:
def handle_quaternion_missing_values(rot_data: np.ndarray) -> np.ndarray:
    """
    Handle missing values in quaternion data intelligently
    
    Key insight: Quaternions must have unit length |q| = 1
    If one component is missing, we can reconstruct it from the others
    """
    rot_cleaned = rot_data.copy()
    
    for i in range(len(rot_data)):
        row = rot_data[i]
        missing_count = np.isnan(row).sum()
        
        if missing_count == 0:
            # No missing values, normalize to unit quaternion
            norm = np.linalg.norm(row)
            if norm > 1e-8:
                rot_cleaned[i] = row / norm
            else:
                rot_cleaned[i] = [1.0, 0.0, 0.0, 0.0]  # Identity quaternion
                
        elif missing_count == 1:
            # One missing value, reconstruct using unit quaternion constraint
            # |w|² + |x|² + |y|² + |z|² = 1
            missing_idx = np.where(np.isnan(row))[0][0]
            valid_values = row[~np.isnan(row)]
            
            sum_squares = np.sum(valid_values**2)
            if sum_squares <= 1.0:
                missing_value = np.sqrt(max(0, 1.0 - sum_squares))
                # Choose sign for continuity with previous quaternion
                if i > 0 and not np.isnan(rot_cleaned[i-1, missing_idx]):
                    if rot_cleaned[i-1, missing_idx] < 0:
                        missing_value = -missing_value
                rot_cleaned[i, missing_idx] = missing_value
                rot_cleaned[i, ~np.isnan(row)] = valid_values
            else:
                rot_cleaned[i] = [1.0, 0.0, 0.0, 0.0]
        else:
            # More than one missing value, use identity quaternion
            rot_cleaned[i] = [1.0, 0.0, 0.0, 0.0]
    
    return rot_cleaned

In [9]:
def mirror_quaternion(quat):
    """
    Mirror a single quaternion through the YZ plane.

    Args:
        quat (array of shape (N, 4)): [w, x, y, z]

    Returns:
        mirrored (np.ndarray of shape (N, 4)): mirrored quaternion [w, x, y, z]
    """

    P = np.diag([-1, 1, 1])  # reflection through YZ
    rot = R.from_quat(quat[:, [1, 2, 3, 0]])  # SciPy uses [x, y, z, w]
    R_mat = rot.as_matrix()
    R_flipped = P @ R_mat @ P
    flipped = R.from_matrix(R_flipped).as_quat()
    return flipped[:, [3, 0, 1, 2]]  # back to [w, x, y, z]


def mirror_data(data):
    """
    Mirror left-handed samples to match right-handed frame.

    Args:
        data (np.ndarray of shape (N, 7)): sensor data
    
    Returns:
        A new array with mirrored left-handed samples.

    """
    
    data[:, 0] = -data[:, 0]
    data[:, 3:] = mirror_quaternion(data[:, 3:]) # [w, x, y, z]

    return data

def process_left_handed(df, dem):
    
    left_handed = dem[dem["handedness"] == 0]
    left_handed = df.loc[df["subject"].isin(left_handed["subject"])]
    cols_to_transform = ["acc_x", "acc_y", "acc_z", "rot_w", "rot_x", "rot_y", "rot_z"]
    left_handed_arr = left_handed[cols_to_transform].to_numpy()
    df.loc[df["subject"].isin(left_handed["subject"]), cols_to_transform] = mirror_data(left_handed_arr)
    return df

In [10]:
def add_features(df: pd.DataFrame) -> pd.DataFrame:
    df["acc_mag"] = np.linalg.norm(df[Config.ACC_COLS].values, axis=1)
    df["acc_mag_jerk"] = df.groupby("sequence_id")["acc_mag"].diff().fillna(0)
    df["jerk_x"], df["jerk_y"], df["jerk_z"] = np.gradient(df["acc_x"]), np.gradient(df["acc_y"]), np.gradient(df["acc_z"])
    df["jerk_magnitude"] = np.linalg.norm(df[["jerk_x", "jerk_y", "jerk_z"]].values, axis=1)

    window = 20
    for _, g in df.groupby("sequence_id"):
        df.loc[g.index, "acc_xy_corr"] = g["acc_x"].rolling(window, min_periods=1).corr(g["acc_y"]).fillna(0)
        df.loc[g.index, "acc_xz_corr"] = g["acc_x"].rolling(window, min_periods=1).corr(g["acc_z"]).fillna(0)
        df.loc[g.index, "acc_yz_corr"] = g["acc_y"].rolling(window, min_periods=1).corr(g["acc_z"]).fillna(0)

    df["rot_angle"] = 2 * np.arccos(df["rot_w"].clip(-1, 1))
    df["rot_angle_vel"] = df.groupby("sequence_id")["rot_angle"].diff().fillna(0)

    rot_numpy = df[Config.ROT_COLS].to_numpy()
    angular_vel = calculate_angular_velocity_from_quat(rot_numpy)
    angular_dist = calculate_angular_distance(rot_numpy)
    df[["angular_vel_x", "angular_vel_y", "angular_vel_z"]] = angular_vel
    df["angular_distance"] = angular_dist
    df["angular_vel_magnitude"] = np.linalg.norm(angular_vel, axis=1)

    linear_accel = remove_gravity_from_acc(df[Config.ACC_COLS], df[Config.ROT_COLS])
    df[["acc_x2", "acc_y2", "acc_z2"]] = linear_accel
    df["acc_mag2"] = np.linalg.norm(linear_accel, axis=1)
    df["acc_mag_jerk2"] = df.groupby("sequence_id")["acc_mag2"].diff().fillna(0)
    df["jerk_x2"], df["jerk_y2"], df["jerk_z2"] = np.gradient(df["acc_x2"]), np.gradient(df["acc_y2"]), np.gradient(df["acc_z2"])
    df["jerk_magnitude2"] = np.linalg.norm(df[["jerk_x2", "jerk_y2", "jerk_z2"]].values, axis=1)

    for _, g in df.groupby("sequence_id"):
        df.loc[g.index, "acc_xy_corr2"] = g["acc_x2"].rolling(window, min_periods=1).corr(g["acc_y2"]).fillna(0)
        df.loc[g.index, "acc_xz_corr2"] = g["acc_x2"].rolling(window, min_periods=1).corr(g["acc_z2"]).fillna(0)
        df.loc[g.index, "acc_yz_corr2"] = g["acc_y2"].rolling(window, min_periods=1).corr(g["acc_z2"]).fillna(0)

    df.replace([np.inf, -np.inf], 0, inplace=True)
    df.fillna(0, inplace=True)
    return df

def pad_sequences(sequences: list, maxlen: int, padding: str="pre", truncating: str="pre", dtype: str="float32") -> np.ndarray:
    n_samples = len(sequences)
    sample_shape = tuple()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break
    x = np.zeros((n_samples, maxlen) + sample_shape, dtype=dtype)
    for idx, s in enumerate(sequences):
        if len(s) == 0:
            continue
        s_np = np.asarray(s, dtype=dtype)
        if truncating == "pre":
            s_np = s_np[-maxlen:]
        else:
            s_np = s_np[:maxlen]
        trunc = s_np.shape[0]
        if padding == "pre":
            x[idx, -trunc:] = s_np
        else:
            x[idx, :trunc] = s_np
    return x

def to_categorical(y, num_classes=None):
    y = np.array(y, dtype="int")
    if not num_classes:
        num_classes = np.max(y) + 1
    return np.eye(num_classes)[y]

def preprocess_left_handed_pd(df_pd: pd.DataFrame, demo_pd: pd.DataFrame) -> pd.DataFrame:
    pl_df = pl.DataFrame(df_pd)
    pl_demo = pl.DataFrame(demo_pd)
    l_subjs = pl_demo.filter(pl.col("handedness") == 0)["subject"].to_list()
    r_tr = pl_df.filter(~pl.col("subject").is_in(l_subjs))
    l_tr = pl_df.filter(pl.col("subject").is_in(l_subjs))
    if l_tr.shape[0] == 0:
        return r_tr.to_pandas()

    l_tr = l_tr.with_columns(-pl.col("acc_y"))
    l_tr = l_tr.with_columns(-pl.col("acc_z"))
    rot_np = l_tr.select(Config.ROT_COLS).to_numpy()
    rot_scipy = rot_np[:, [1, 2, 3, 0]]
    r = R.from_quat(rot_scipy)
    tmp = r.as_euler("xyz")
    tmp[:, 1] = -tmp[:, 1]
    tmp[:, 2] = -tmp[:, 2]
    r = R.from_euler("xyz", tmp)
    tmp = r.as_quat()
    l_tr = l_tr.with_columns(pl.DataFrame(tmp, schema=["rot_x", "rot_y", "rot_z", "rot_w"]))
    pl_df2 = pl.concat([r_tr, l_tr]).sort(by="row_id")
    return pl_df2.to_pandas()

In [11]:
def extract_comprehensive_features(sequence: pl.DataFrame, demographics: pl.DataFrame) -> pd.DataFrame:
    """
    Extract features from IMU data with world acceleration transformation
    
    Feature Groups:
    1. Device Acceleration (acc_x, acc_y, acc_z) - raw sensor data
    2. Rotation Quaternion (rot_w, rot_x, rot_y, rot_z) - device orientation  
    3. World Acceleration (NEW) - orientation-normalized acceleration
    4. Demographics - subject characteristics
    5. Sequence metadata - length, etc.
    """
    seq_df = sequence.to_pandas()
    demo_df = demographics.to_pandas()
    
    # Convert to pandas for processing
    seq_df = sequence.to_pandas()
    demo_df = demographics.to_pandas()
    
    rot_data = seq_df[Config.ROT_COLS].copy()
    rot_data = rot_data.ffill().bfill()
    rot_data_clean = handle_quaternion_missing_values(rot_data.values)
    
    seq_df[Config.ROT_COLS] = rot_data_clean
    
    if len(demo_df) > 0 and demo_df.iloc[0].get("handedness", 1) == 0:
        seq_df = process_left_handed(seq_df, demo_df)
   

     # Handle missing values in basic sensor data
    acc_data = seq_df[Config.ACC_COLS].copy()
    acc_data = acc_data.ffill().bfill().fillna(0)

    rot_data = seq_df[Config.ROT_COLS].copy()
    rot_data = rot_data.ffill().bfill()
    rot_data_clean = handle_quaternion_missing_values(rot_data.values)

    # rot_data_clean = seq_df[Config.ROT_COLS].copy()

    #linear acc
    try:
        acc_gravity_removed = remove_gravity_from_acc(acc_data.values, rot_data_clean)
        # print("✓ Gravity remove successfully")  # Reduced verbosity
    except Exception as e:
        print(f"Warning: Gravity remove failed: {e}")
        acc_gravity_removed = acc_data.values.copy()  # Fallback to device coordinates

    #linear world acc
    try:
        world_acc_data = compute_world_acceleration(acc_gravity_removed, rot_data_clean)
        # print("✓ World acceleration computed successfully")  # Reduced verbosity
    except Exception as e:
        print(f"Warning: World acceleration computation failed: {e}")
        world_acc_data = acc_gravity_removed.values.copy()  # Fallback to device coordinates  

    #angular velocity
    try:
        angular_velocity = calculate_angular_velocity_from_quat(rot_data_clean, time_delta=1/200)
        # print("✓ Calculate angular velocity successfully")  # Reduced verbosity
    except Exception as e:
        print(f"Warning: Calculate angular velocity failed: {e}")
        angular_velocity = rot_data.values.copy()  # Fallback to device coordinates

    #angular distance
    try:
        angular_distance = calculate_angular_distance(rot_data_clean)
        # print("✓ Calculate angular distance successfully")  # Reduced verbosity
    except Exception as e:
        print(f"Warning: Calculate angular distance failed: {e}")
        angular_distance = rot_data.values.copy()  # Fallback to device coordinates
    
    # Initialize feature dictionary
    features = {}
    
    # Add sequence metadata
    features['sequence_length'] = len(seq_df)
    
    # Add demographics features
    if len(demo_df) > 0:
        demo_row = demo_df.iloc[0]
        features['age'] = demo_row.get('age', 0)
        features['adult_child'] = demo_row.get('adult_child', 0)
        features['sex'] = demo_row.get('sex', 0)
        features['handedness'] = demo_row.get('handedness', 0)
        features['height_cm'] = demo_row.get('height_cm', 0)
        features['shoulder_to_wrist_cm'] = demo_row.get('shoulder_to_wrist_cm', 0)
        features['elbow_to_wrist_cm'] = demo_row.get('elbow_to_wrist_cm', 0)
    
    # Define feature arrays for statistical extraction
    feature_arrays = {
        'acc': acc_data.values,           # Device acceleration (3D)
        'rot': rot_data_clean,            # Rotation quaternion (4D) 
        'world_acc': world_acc_data,      # World acceleration (3D) - KEY INNOVATION
        'acc_g_remove': acc_gravity_removed,
        'ang_vel': angular_velocity,
        'ang_dis': angular_distance,
    }
    
    # Extract statistical features for each data source
    for source_name, array in feature_arrays.items():
        if array.ndim == 1:
            array = array.reshape(-1, 1)
        
        n_features = array.shape[1]
        
        for feat_idx in range(n_features):
            feat_data = array[:, feat_idx]
            
            # Create feature name
            if source_name == 'acc':
                axis_names = ['x', 'y', 'z']
                prefix = f"acc_{axis_names[feat_idx]}"
            elif source_name == 'rot':
                comp_names = ['w', 'x', 'y', 'z']
                prefix = f"rot_{comp_names[feat_idx]}"
            elif source_name == 'world_acc':
                axis_names = ['x', 'y', 'z']  
                prefix = f"world_acc_{axis_names[feat_idx]}"
            else:
                prefix = f"{source_name}_{feat_idx}" if n_features > 1 else source_name
            
            # Extract comprehensive statistical features
            features.update(extract_statistical_features(feat_data, prefix))
    
    # Compute magnitude features (important for motion intensity)
    acc_magnitude = np.linalg.norm(acc_data.values, axis=1)
    features.update(extract_statistical_features(acc_magnitude, 'acc_magnitude_raw'))

    # Compute linear_acc_magnitude
    linear_acc_magnitude = np.linalg.norm(acc_gravity_removed, axis=1)
    features.update(extract_statistical_features(linear_acc_magnitude, 'linear_acc_magnitude'))

    world_acc_magnitude = np.linalg.norm(world_acc_data, axis=1)
    features.update(extract_statistical_features(world_acc_magnitude, 'world_acc_magnitude'))
    
    # Cross-feature: difference between device and world acceleration magnitudes
    # This captures how much device orientation affects motion measurement
    acc_world_diff = linear_acc_magnitude - world_acc_magnitude
    features.update(extract_statistical_features(acc_world_diff, 'acc_world_diff'))
    
    # Convert to DataFrame
    result_df = pd.DataFrame([features])
    
    # Handle any remaining NaN values
    result_df = result_df.fillna(0)
    
    return result_df

In [12]:
def extract_statistical_features(data: np.ndarray, prefix: str) -> dict:
    """
    Extract comprehensive statistical features from a 1D time series
    
    Returns features that capture:
    - Central tendency: mean, median, mode region
    - Spread: std, variance, range, IQR  
    - Shape: skewness, kurtosis
    - Dynamics: differences, trends, changes
    - Segments: beginning vs middle vs end behavior
    """
    
    features = {}
    
    # Basic statistics
    features[f'{prefix}_mean'] = np.mean(data)
    features[f'{prefix}_std'] = np.std(data)
    features[f'{prefix}_var'] = np.var(data)
    features[f'{prefix}_min'] = np.min(data)
    features[f'{prefix}_max'] = np.max(data)
    features[f'{prefix}_median'] = np.median(data)
    features[f'{prefix}_q25'] = np.percentile(data, 25)
    features[f'{prefix}_q75'] = np.percentile(data, 75)
    features[f'{prefix}_iqr'] = np.percentile(data, 75) - np.percentile(data, 25)
    features[f'{prefix}_mean_abs'] = np.abs(data).mean()
    features[f'{prefix}_rms'] = np.sqrt((data**2).mean())
    
    # Range and boundary features
    features[f'{prefix}_range'] = np.max(data) - np.min(data)
    features[f'{prefix}_first'] = data[0] if len(data) > 0 else 0
    features[f'{prefix}_last'] = data[-1] if len(data) > 0 else 0
    features[f'{prefix}_delta'] = data[-1] - data[0] if len(data) > 0 else 0
    
    # Higher order moments (shape of distribution)
    if len(data) > 1 and np.std(data) > 1e-8:
        features[f'{prefix}_skew'] = pd.Series(data).skew()
        features[f'{prefix}_kurt'] = pd.Series(data).kurtosis()
    else:
        features[f'{prefix}_skew'] = 0
        features[f'{prefix}_kurt'] = 0
    
    # Differential features (capture dynamics)
    if len(data) > 1:
        diff_data = np.diff(data)
        features[f'{prefix}_diff_mean'] = np.mean(diff_data)
        features[f'{prefix}_diff_std'] = np.std(diff_data)
        features[f'{prefix}_n_changes'] = np.sum(np.abs(diff_data) > np.std(data) * 0.1)  # Significant changes
    else:
        features[f'{prefix}_diff_mean'] = 0
        features[f'{prefix}_diff_std'] = 0
        features[f'{prefix}_n_changes'] = 0
    
    # Correlation with time (trend detection)
    if len(data) > 2:
        time_indices = np.arange(len(data))
        try:
            corr_coef = np.corrcoef(time_indices, data)[0, 1]
            features[f'{prefix}_time_corr'] = corr_coef if not np.isnan(corr_coef) else 0
        except:
            features[f'{prefix}_time_corr'] = 0
    else:
        features[f'{prefix}_time_corr'] = 0
    
    # Segment features (beginning, middle, end patterns)
    seq_len = len(data)
    if seq_len >= 9:  # Need sufficient data for meaningful segments
        seg_size = seq_len // 3
        seg1 = data[:seg_size]           # Beginning (Transition phase)
        seg2 = data[seg_size:2*seg_size] # Middle (Pause phase)  
        seg3 = data[2*seg_size:]         # End (Gesture phase)
        
        features[f'{prefix}_seg1_mean'] = np.mean(seg1)
        features[f'{prefix}_seg2_mean'] = np.mean(seg2)
        features[f'{prefix}_seg3_mean'] = np.mean(seg3)
        
        features[f'{prefix}_seg1_std'] = np.std(seg1)
        features[f'{prefix}_seg2_std'] = np.std(seg2)
        features[f'{prefix}_seg3_std'] = np.std(seg3)
        
        # Segment transitions (important for distinguishing gesture types)
        features[f'{prefix}_seg1_to_seg2'] = np.mean(seg2) - np.mean(seg1)
        features[f'{prefix}_seg2_to_seg3'] = np.mean(seg3) - np.mean(seg2)
    else:
        # Not enough data for meaningful segments
        for seg in [1, 2, 3]:
            features[f'{prefix}_seg{seg}_mean'] = features[f'{prefix}_mean']
            features[f'{prefix}_seg{seg}_std'] = features[f'{prefix}_std']
        features[f'{prefix}_seg1_to_seg2'] = 0
        features[f'{prefix}_seg2_to_seg3'] = 0

    if len(data) > 1:
        series = pd.Series(data)

        # Peak detection - count significant changes
        features[f"{prefix}_peak_count"] = (series.diff().abs() > (series.std() * 0.5)).sum()

        # Movement intensity - mean absolute deviation (MAD)
        features[f"{prefix}_mad"] = (series - series.mean()).abs().mean()

        # Range normalization - signal range relative to std
        features[f"{prefix}_range_norm"] = (series.max() - series.min()) / (series.std() + 1e-8)

        # Acceleration analysis - second derivative
        accel = series.diff().diff()
        features[f"{prefix}_accel_mean"] = accel.abs().mean()
        features[f"{prefix}_accel_std"] = accel.std()

        # Movement consistency - coefficient of variation (CV)
        features[f"{prefix}_cv"] = series.std() / (abs(series.mean()) + 1e-8)

        # Energy measure - sum of squared values
        features[f"{prefix}_energy"] = (series ** 2).sum()
    else:
        features[f"{prefix}_peak_count"] = 0
        features[f"{prefix}_mad"] = 0
        features[f"{prefix}_range_norm"] = 0
        features[f"{prefix}_accel_mean"] = 0
        features[f"{prefix}_accel_std"] = 0
        features[f"{prefix}_cv"] = 0
        features[f"{prefix}_energy"] = 0
    
    return features

In [13]:
def create_prediction_function(models, feature_cols, imu_cols):
    """Create prediction function for Kaggle evaluation"""
    
    def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
        """
        Prediction function for Kaggle evaluation
        Uses ensemble of trained LightGBM models
        """
        try:
            # Filter sequence to only include IMU columns that we trained with
            available_cols = sequence.columns
            sequence_imu_cols = [col for col in imu_cols if col in available_cols]
            sequence_filtered = sequence.select(pl.col(sequence_imu_cols))
            
            # Extract features using the same method as training
            features = extract_comprehensive_features(sequence_filtered, demographics)
            
            # Ensure we have the same features as training
            missing_features = [col for col in feature_cols if col not in features.columns]
            if missing_features:
                print(f"Warning: Missing features {missing_features}, filling with zeros")
                for col in missing_features:
                    features[col] = 0
            
            X_pred = features[feature_cols]
            print(X_pred)
            
            # Get predictions from all models
            # predictions = []
            # for model in models:
            #     pred_probs = model.predict_proba(X_pred)
            #     pred_class = np.argmax(pred_probs, axis=1)[0]
            #     predictions.append(pred_class)
            
            # # Ensemble prediction (majority vote)
            # final_prediction = max(set(predictions), key=predictions.count)
            
            # # Convert back to gesture name
            # gesture_name = REVERSE_GESTURE_MAPPER[final_prediction]
            probs_all = []
            for model in models:
                pred_probs = model.predict_proba(X_pred)
                probs_all.append(pred_probs[0])  # each is shape (num_classes,)
            
            # Convert to numpy array for averaging
            probs_all = np.array(probs_all)  # shape: (n_models, n_classes)
            
            # 🔥 Soft voting: average probabilities across models
            probs_mean = np.mean(probs_all, axis=0)
            
            # Final predicted class
            final_prediction = int(np.argmax(probs_mean))
            
            # Convert back to gesture name
            gesture_name = REVERSE_GESTURE_MAPPER[final_prediction]
            
            print(f"Predicted: {gesture_name} (class {final_prediction})")
            return gesture_name
            
        except Exception as e:
            print(f"Prediction error: {e}")
            import traceback
            traceback.print_exc()
            return 'Text on phone'  # Fallback prediction
    
    return predict

In [14]:
# def create_prediction_function(
#     models,
#     feature_cols,
#     imu_cols,
#     lstm_model_paths,     # list of 5 model paths
#     prep_paths,           # list of 5 prep paths
#     device,
#     model_cls=IMUCrossAttentionModel
# ):
#     """
#     Create prediction function for hybrid XGBoost + 5-Fold LSTM ensemble.
#     """

#     import torch
#     import numpy as np
#     import pandas as pd
#     import polars as pl
#     import joblib

#     # === Load all 5 LSTM models + preprocessors ===
#     lstm_models = []
#     for model_path, prep_path in zip(lstm_model_paths, prep_paths):
#         prep = joblib.load(prep_path)
#         # scaler = prep["transformer"]
#         pad_len = prep["max_length"]
#         n_classes = len(prep["categories"])
#         imu_features = prep["features"]

#         model = model_cls(len(imu_features), n_classes).to(device)
#         model.load_state_dict(torch.load(model_path, map_location=device))
#         model.eval()
#         lstm_models.append((model, pad_len, imu_features))

#     # === Define the predict function ===
#     def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
#         try:
#             # === 1️⃣ Extract static features ===
#             available_cols = sequence.columns
#             sequence_imu_cols = [col for col in imu_cols if col in available_cols]
#             sequence_filtered = sequence.select(pl.col(sequence_imu_cols))
#             static_features = extract_comprehensive_features(sequence_filtered, demographics)

#             # === 2️⃣ Get embeddings from all 5 LSTM folds ===
#             embeddings = []
#             for model, pad_len, imu_features in lstm_models:
#                 # Convert to pandas for preprocessing
#                 df_seq = sequence_filtered.to_pandas()
#                 df_seq = preprocess_left_handed_pd(df_seq, demographics)
#                 df_seq = add_features(df_seq)

#                 # Scale and pad
#                 # df_seq.loc[:, imu_features] = scaler.transform(df_seq[imu_features])
#                 X_seq = pad_sequences([df_seq[imu_features].to_numpy()], maxlen=pad_len)
#                 X_seq_tensor = torch.tensor(X_seq, dtype=torch.float32).to(device)

#                 # Extract embeddings
#                 with torch.no_grad():
#                     emb = model.extract_features(X_seq_tensor).cpu().numpy().flatten()
#                 embeddings.append(emb)

#             # Average embeddings across folds
#             emb_mean = np.mean(embeddings, axis=0)
#             emb_df = pd.DataFrame([emb_mean], columns=[f"emb_{i}" for i in range(len(emb_mean))])

#             # === 3️⃣ Merge static + embedding features ===
#             # Select static features used in training
#             # X_pred = static_features[feature_cols]

#             # # 🔹 Add embeddings to X_pred
#             # X_pred = pd.concat([X_pred.reset_index(drop=True), emb_df.reset_index(drop=True)], axis=1)
#             X_pred = pd.DataFrame(columns=feature_cols)
#             # create one row of zeros
#             X_pred.loc[0] = 0.0

#             # Fill in static features (if present)
#             # static_features is a DataFrame with one row
#             for col in static_features.columns:
#                 if col in X_pred.columns:
#                     # take first element (static_features is per-sequence)
#                     try:
#                         X_pred.at[0, col] = static_features[col].iloc[0]
#                     except Exception:
#                         # fallback if static_features has scalar or different layout
#                         X_pred.at[0, col] = static_features[col].values[0]

#             # Fill embedding columns (emb_df is one-row)
#             for col in emb_df.columns:
#                 if col in X_pred.columns:
#                     X_pred.at[0, col] = emb_df[col].iloc[0]
#                 else:
#                     # if embedding columns are not in feature_cols (shouldn't happen),
#                     # add them (optional)
#                     X_pred[col] = emb_df[col].iloc[0]

#             # Now X_pred has exact columns and order as feature_cols (plus possibly extra emb cols)
#             # If you want to force exact ordering and drop any extras:
#             X_pred = X_pred[[c for c in feature_cols if c in X_pred.columns]]


#             # === 4️⃣ Predict with XGBoost ensemble ===
#             preds = []
#             for m in models:
#                 probs = m.predict_proba(X_pred)
#                 preds.append(np.argmax(probs, axis=1)[0])

#             final_pred = max(set(preds), key=preds.count)
#             gesture_name = REVERSE_GESTURE_MAPPER[final_pred]

#             print(f"Predicted: {gesture_name} (class {final_pred})")
#             return gesture_name

#         except Exception as e:
#             print(f"❌ Prediction error: {e}")
#             import traceback
#             traceback.print_exc()
#             return "Text on phone"  # fallback

#     return predict


In [15]:
def main():
    lstm_model_paths = [f"{Config.EXPORT_DIR}/model_best_fold_{i}.pt" for i in range(1, 5)]
    prep_paths = [f"{Config.EXPORT_DIR}/prep_fold_{i}.joblib" for i in range(1, 5)]
    model = joblib.load("/kaggle/input/xgboost1/xgb_model.pkl")
    feature_cols = joblib.load("/kaggle/input/xgboost1/feature_cols.pkl")
    imu_cols = joblib.load("/kaggle/input/xgboost1/imu_cols.pkl")

    for m in model:
        if hasattr(m, "get_booster"):   # XGBClassifier / XGBRegressor wrapper
            m.get_booster().set_param({"device": "cpu"})
        elif hasattr(m, "set_param"):   # raw Booster
            m.set_param({"device": "cpu"})
        elif hasattr(m, "set_params"):  # sklearn-style wrapper
            m.set_params(device="cpu")
    
    # Create prediction function
    # predict_func = create_prediction_function(model, feature_cols, imu_cols, lstm_model_paths, prep_paths,"cpu")
    predict_func = create_prediction_function(model, feature_cols, imu_cols)

    print(f"✓ Models ready for inference")
    
    return predict_func, model

# Execute main pipeline
if __name__ == "__main__":
    predict_function, trained_models = main()
    
    # Setup inference server
    print("\nSetting up inference server...")
    inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict_function)
    
    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    # if 0 :
        inference_server.serve()
    else:
        # Local testing
        inference_server.run_local_gateway(
            data_paths=(
                Config.TEST_PATH,
                Config.TEST_DEMOGRAPHICS_PATH,
            )
        )

✓ Models ready for inference

Setting up inference server...
   sequence_length  age  adult_child  sex  handedness  height_cm  \
0               56   13            0    0           1      177.0   

   shoulder_to_wrist_cm  elbow_to_wrist_cm  acc_x_mean  acc_x_std  ...  \
0                    52               27.0    7.750837   1.041217  ...   

   acc_world_diff_seg3_std  acc_world_diff_seg1_to_seg2  \
0             2.580307e-15                 1.973730e-16   

   acc_world_diff_seg2_to_seg3  acc_world_diff_peak_count  acc_world_diff_mad  \
0                -5.921189e-17                         34        2.383581e-15   

   acc_world_diff_range_norm  acc_world_diff_accel_mean  \
0                   0.000001               4.671161e-15   

   acc_world_diff_accel_std  acc_world_diff_cv  acc_world_diff_energy  
0              6.230048e-15       2.828295e-07           5.301145e-28  

[1 rows x 764 columns]
Predicted: Eyebrow - pull hair (class 2)
   sequence_length  age  adult_child  sex  